Self check to write a small delta table and then read it from a new spark session.

In [1]:
result = {
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": False,
    "message": "",
    "plugin": "spark",
}

In [2]:
import json

from freeds.spark import get_spark_session, show_spark_info

try:
    spark = get_spark_session("self-check1")
    db_name = "selfcheck_db"
    table_name = f"{db_name}.selfcheck_tbl"

    # create some data in delta
    print(f"Dropping and recreating database {db_name}")
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Writing table {table_name}")
    data = spark.range(100)
    (
        data.write.option(  # .mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
            "mergeSchema", "true"
        )
        .format("iceberg")  # Options: 'parquet', 'csv', 'json', 'orc', 'iceberg', 'delta' etc.
        .saveAsTable(table_name)
    )
    spark.stop()

    # read some data in delta
    spark = get_spark_session("self-check2")
    data = spark.table(table_name)
    show_spark_info(spark)
    data.show(5)

    # clean up
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.stop()

    result["message"] = "Executed spark cell ok."
    result["passed"] = True

except Exception as ex:
    result["message"] = str(ex)
    print(str(ex))
    result["passed"] = False

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df109be3-1d02-4714-8202-3e36be448181;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.3.0/delta-spark_2.12-3.3.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.3.0!delta-spark_2.12.jar (2448ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.3.0/delta-storage-3.3.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.3.0!delta-storage.jar (133ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (159ms)
:: resolution report :: resolve 3633ms :: 

Dropping and recreating database selfcheck_db
Writing table selfcheck_db.selfcheck_tbl
An error occurred while calling o46.saveAsTable.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:568)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at java.base/jdk.internal.reflect

In [3]:
print(json.dumps(result, indent=4))

{
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": false,
    "message": "An error occurred while calling o46.saveAsTable.\n: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.\n\tat org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)\n\tat org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)\n\tat org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)\n\tat org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)\n\tat org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:568)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke